# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
!pip install transformers


[notice] A new release of pip available: 22.3.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install nltk


[notice] A new release of pip available: 22.3.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install torch


[notice] A new release of pip available: 22.3.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
# импортируем библиотеки и модули из них, задаем переменные

import pandas as pd
import torch
import transformers
import numpy as np
import re
import nltk
import warnings
import lightgbm as lgb

from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle, resample
from nltk.stem import WordNetLemmatizer 
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
from tqdm import tqdm

url = 'https://code.s3.yandex.net/datasets/toxic_comments.csv'
warnings.filterwarnings("ignore")
STATE = 17
tqdm.pandas()

In [5]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vk\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
# загружаем данные

data = pd.read_csv(url)

In [7]:
# выводим первые 5 строк

data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [8]:
# выводим общую информацию о датасете

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [9]:
# выводим количество положительных и отрицательных твитов

data['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

In [10]:
# пишем функцию, которая лемматизирует и чистит текст

nltk.download('wordnet')
nltk.download('omw-1.4')

lemmatizer = WordNetLemmatizer()

def lem_clear(text):
  text = text.lower()
  text_list = nltk.word_tokenize(text)
  lemm_text = ' '.join([lemmatizer.lemmatize(text) for text in text_list])
  clear_text = re.sub(r'[^a-zA-Z]', ' ', lemm_text) 
  return " ".join(clear_text.split())

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vk\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\vk\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [11]:
%%time

data['lemm_text'] = data['text'].progress_apply(lem_clear)

100%|██████████| 159292/159292 [04:17<00:00, 618.72it/s]


Wall time: 4min 17s


In [12]:
data.head()

,Unnamed: 0,text,toxic,lemm_text
0,0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits made under my userna...
1,1,D'aww! He matches this background colour I'm s...,0,d aww he match this background colour i m seem...
2,2,"Hey man, I'm really not trying to edit war. It...",0,hey man i m really not trying to edit war it s...
3,3,"""\nMore\nI can't make any real suggestions on ...",0,more i ca n t make any real suggestion on impr...
4,4,"You, sir, are my hero. Any chance you remember...",0,you sir are my hero any chance you remember wh...


In [13]:
# удаляем лишние столбцы

data = data.drop(columns=['Unnamed: 0', 'text'], axis=1)

In [14]:
data.head()

,toxic,lemm_text
0,0,explanation why the edits made under my userna...
1,0,d aww he match this background colour i m seem...
2,0,hey man i m really not trying to edit war it s...
3,0,more i ca n t make any real suggestion on impr...
4,0,you sir are my hero any chance you remember wh...


In [15]:
# делим на таргеты и фичи, а также на обучающую, валидационную и тестовую выборки

target = data['toxic']
features = data.drop(columns=['toxic'], axis=1)

features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.1, random_state=STATE, stratify=target)

print(features.shape)
print(features_train.shape)
print(features_test.shape)

(159292, 1)
(143362, 1)
(15930, 1)


In [16]:
# подгружаем "стопслова" и создаем счетчик

nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vk\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
corpus_train = list(features_train['lemm_text'])
corpus_test = list(features_test['lemm_text'])

In [18]:
features_train = count_tf_idf.fit_transform(corpus_train)
features_test = count_tf_idf.transform(corpus_test)

print(features_test.shape)
print(features_train.shape)

(15930, 151595)
(143362, 151595)


In [19]:
# создаем модель и задаем параметр кросс-валидации

model = LogisticRegression()
cv = 3

In [20]:
# расчитаем F1 score на обучающей выборке без подбора гиперпараметров и с дисбалансом классов
%%time

f1_score_train_ds = cross_val_score(model, features_train, target_train, cv=cv, scoring='f1').mean()
print('F1 score:', round(f1_score_train_ds, 3))

UsageError: Line magic function `%%time` not found.


## Обучение

In [ ]:
pipe_lr = Pipeline([('tfidf', TfidfVectorizer(stop_words=stopwords)),
                    ('model', LogisticRegression(random_state=STATE))])

pipe_rfc = Pipeline([('tfidf', TfidfVectorizer(stop_words=stopwords)),
                     ('model', RandomForestClassifier(random_state=STATE))])

In [ ]:
grid_lr = {'tfidf__ngram_range': [(1, 3)],
           'tfidf__min_df': [3],
           'model__C': np.linspace(0.0001, 100, 20)}

grid_rfc = {'tfidf__ngram_range': [(1, 3)],
            'tfidf__min_df': [3],
            'model__max_depth': [4, 5, 10],
            'model__max_features': [2, 3],
            'model__min_samples_leaf': [3, 4, 5],
            'model__n_estimators': [100, 200, 300]}

In [ ]:
def params_grid(pipe, params):

  grid_search = GridSearchCV(pipe, params, cv=cv, n_jobs=-1, scoring='f1')
  grid_search.fit(corpus_train, target_train)

  best_params = grid_search.best_params_
  best_f1 = grid_search.best_score_

  model = grid_search.best_estimator_

  print('Best params:', best_params)
  print('Best f1 score:', best_f1)
  return model, best_f1

**LogisticRegression**

In [ ]:
%%time

lr_model, lr_f1 = params_grid(pipe_lr, grid_lr)

**RandomForestClassifier**

In [ ]:
%%time

rfc_model, rfc_f1 = params_grid(pipe_rfc, grid_rfc)

**Тестирование лучшей модели**

In [ ]:
lr_pred = lr_model.predict(corpus_test)
print(f1_score(target_test, lr_pred))

## Выводы

- Были загружены и подготовлены данные
- Все твиты были очищены и лемматизированы
- Удалены слова которые встречаются во всем корпусе менее 3 раз, чтобы избавиться от мусора при обучении
- Было обучено 2 модели: LogisticRegression, RandomForestClassifier
- Наилучший F1 score показала модель LigisticRegression 